In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import sys; sys.path.insert(0, '../')

import matplotlib.pyplot as plt

# always import gbm_algos first !
import xgboost, lightgbm, catboost

# Core
import numpy as np
import scipy.io as io
from torch.autograd import grad
from torch.utils.data import DataLoader, Dataset

# Sklearn
from sklearn.ensemble import RandomForestRegressor
from mlens.ensemble import SuperLearner

# Let's do facy optimizers
from optimizers import Lookahead, AdamGC, SGDGC
from madgrad import MADGRAD
from lbfgsnew import LBFGSNew
# Modify at /usr/local/lib/python3.9/site-packages/torch_lr_finder/lr_finder.py
from torch_lr_finder import LRFinder
from onecyclelr import OneCycleLR
import pcgrad
from pytorch_stats_loss import torch_wasserstein_loss, torch_energy_loss
from geomloss import SamplesLoss
from utils import *
from models import RobustPCANN

# Model selection
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge
from pde_diff import TrainSTRidge, FiniteDiff, print_pde
from RegscorePy.bic import bic

from tqdm import trange

from pytorch_robust_pca import *

[MLENS] backend: threading


[pyKeOps]: Warning, no cuda detected. Switching to cpu only.
Running Python 3.9.7
You can use npar for np.array


In [2]:
DATA_PATH = "../experimental_data/burgers_shock.mat"
data = io.loadmat(DATA_PATH)

# Adding noise
noise_intensity = 0.01
noisy_xt = True

t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T

if noise_intensity>0.0:
    Exact = perturb(Exact, intensity=noise_intensity, noise_type="normal")
    print("Perturbed Exact with intensity =", float(noise_intensity))
else: print("Clean Exact")

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact.flatten()[:,None]              

# Doman bounds
lb = X_star.min(0)
ub = X_star.max(0)

N = 2000; include_unsup = False
print(f"Training with {N} samples")
idx = np.random.choice(X_star.shape[0], N, replace=False)
X_u_train = X_star[idx, :]
u_train = u_star[idx,:]

if noisy_xt and noise_intensity>0.0:
    print("Noisy (x, t)")
    X_u_train = perturb(X_u_train, intensity=noise_intensity, noise_type="normal")
else: print("Clean (x, t)")

# Unsup data
N_res = N
idx_res = np.array(range(X_star.shape[0]-1))[~idx]
idx_res = np.random.choice(idx_res.shape[0], N_res, replace=True)
X_res = X_star[idx_res, :]
if include_unsup:
    print(f"Training with {N_res} unsup samples")
    X_u_train = np.vstack([X_u_train, X_res])

# Convert to torch.tensor
X_u_train = torch.tensor(X_u_train).float().requires_grad_(True)
u_train = torch.tensor(u_train).float().requires_grad_(True)
X_star = torch.tensor(X_star).float().requires_grad_(True)
u_star = torch.tensor(u_star).float().requires_grad_(True)

# lb and ub are used in adversarial training
scaling_factor = 1.0
lb = scaling_factor*to_tensor(lb, False)
ub = scaling_factor*to_tensor(ub, False)

feature_names=('uf', 'u_x', 'u_xx', 'u_xxx')

Perturbed Exact with intensity = 0.01
Training with 2000 samples
Noisy (x, t)


In [3]:
class DenoisingNetwork(nn.Module):
    def __init__(self, model, init_cs=(0.5, 0.5), init_betas=(0.0, 0.0)):
        super(DenoisingNetwork, self).__init__()
        # pls init the self.model before
        self.model = model
        
        # FFTNN
        self.in_fft_nn = FFTTh(c=init_cs[0])
        self.out_fft_nn = FFTTh(c=init_cs[1])

        # Robust Beta-PCA
        self.inp_rpca = RobustPCANN(beta=init_betas[0], is_beta_trainable=True, inp_dims=2, hidden_dims=32)
        self.out_rpca = RobustPCANN(beta=init_betas[1], is_beta_trainable=True, inp_dims=1, hidden_dims=32)
        
    def forward(self, X_input, X_input_noise, y_input, y_input_noise):
        X_input, y_input = self.denoise(X_input, X_input_noise, y_input, y_input_noise)
        return self.model(X_input), y_input
    
    def denoise(self, X_input, X_input_noise, y_input, y_input_noise):
        # Denoising process
        
        # (1) Denoising FFT on (x, t)
        # This line returns the approx. recon.
        X_input_noise = cat(torch.fft.ifft(self.in_fft_nn(X_input_noise[1])*X_input_noise[0]).real.reshape(-1, 1), 
                            torch.fft.ifft(self.in_fft_nn(X_input_noise[3])*X_input_noise[2]).real.reshape(-1, 1))
        X_input_noise = X_input-X_input_noise
        X_input = self.inp_rpca(X_input, X_input_noise, normalize=True)

        # (2)Denoising FFT on y_input
        y_input_noise = y_input-torch.fft.ifft(self.out_fft_nn(y_input_noise[1])*y_input_noise[0]).real.reshape(-1, 1)
        y_input = self.out_rpca(y_input, y_input_noise, normalize=True)
        
        return X_input, y_input
    
    def loss(self, v1, v2):
        return F.mse_loss(v1, v2, reduction='mean')
    
    def get_selector_data(self, x, t):
        uf = self.model(cat(x, t))
        u_t = self.gradients(uf, t)[0]
        
        ### PDE Loss calculation ###
        u_x = self.gradients(uf, x)[0]
        u_xx = self.gradients(u_x, x)[0]
        u_xxx = self.gradients(u_xx, x)[0]
        
        return cat(uf, u_x, u_xx, u_xxx), u_t
    
    def gradients(self, func, x):
        return grad(func, x, create_graph=True, retain_graph=True, grad_outputs=torch.ones(func.shape))

In [4]:
_, x_fft, x_PSD = fft1d_denoise(X_u_train[:, 0:1], c=-5, return_real=True)
_, t_fft, t_PSD = fft1d_denoise(X_u_train[:, 1:2], c=-5, return_real=True)
_, u_train_fft, u_train_PSD = fft1d_denoise(u_train, c=-5, return_real=True)

In [5]:
solver = DenoisingNetwork(model=TorchMLP(dimensions=[2, 50, 50, 50 ,50, 50, 1], activation_function=nn.Tanh, bn=None, dropout=None))

Using old implementation of TorchMLP. See models.py for more new model-related source code.


In [6]:
lets_pretrain = True

In [7]:
if lets_pretrain:
    print("Pretraining...")
    pretraining_optimizer = LBFGSNew(solver.parameters(), 
                                     lr=1e-1, max_iter=300, 
                                     max_eval=int(300*1.25), history_size=150, 
                                     line_search_fn=True, batch_mode=False)

    solver.train()
    for i in range(200):
        def pretraining_closure():
            global N, X_u_train, u_train
            if torch.is_grad_enabled(): pretraining_optimizer.zero_grad()
            preds, adjusted_grounds = solver(X_u_train, (x_fft, x_PSD, t_fft, t_PSD), 
                                  u_train, (u_train_fft, u_train_PSD))
            mse_loss = solver.loss(preds, adjusted_grounds)
            if mse_loss.requires_grad: mse_loss.backward(retain_graph=True)
            return mse_loss

        pretraining_optimizer.step(pretraining_closure)

        if (i % 100) == 0:
            l = pretraining_closure()
            curr_loss = l.item()
            print("Epoch {}: ".format(i), curr_loss)

Pretraining...
Epoch 0:  0.001017270958982408
Epoch 100:  0.00010209782340098172


In [8]:
# # Fine-tuning the solver network
# f_opt = torch.optim.LBFGS(semisup_model.network.parameters(), lr=1e-1, max_iter=300, max_eval=int(1.25*300), history_size=300)

# def finetuning_closure():
#     global N, X_train, u_train
#     if torch.is_grad_enabled(): f_opt.zero_grad()
#     # the solver network only consider the first N samples.
#     loss = F.mse_loss(semisup_model.network(*dimension_slicing(X_u_train[:N, :])), u_train[:N, :])
#     if loss.requires_grad: loss.backward(retain_graph=True)
#     return loss

# semisup_model.network.train()
# semisup_model.selector.eval()

# for i in range(50):
#     f_opt.step(finetuning_closure)
#     if i%10==0:
#         loss = finetuning_closure()
#         print(loss.item())

In [9]:
n_test = min(50000, X_star.shape[0])
idx_test = np.arange(n_test)
referenced_derivatives, u_t = solver.get_selector_data(*dimension_slicing(X_star[idx_test, :]))

In [10]:
referenced_derivatives = to_numpy(referenced_derivatives); u_t = to_numpy(u_t)

alpha = 1
const_range = (-1.5, 1.5)

X_input = referenced_derivatives
y_input = u_t

poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
X_input = poly.fit_transform(X_input)

poly_feature_names = poly.get_feature_names(feature_names)
for i, f in enumerate(poly_feature_names):
    poly_feature_names[i] = f.replace(" ", "*")

In [20]:
# Set normalize=1
w = TrainSTRidge(X_input[:, :], y_input, 1e-6, d_tol=2)
print("PDE derived using STRidge")
print_pde(w, poly_feature_names[:])

PDE derived using STRidge
u_t = (-0.000021 +0.000000i)u_xx
    + (-0.011102 +0.000000i)uf*u_x
   
